In [1]:
import gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import re
import copy
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
env = gym.make("SpaceInvaders-v0")

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/avmelkote/.local/lib/python3.9/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment SpaceInvaders-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [2]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [3]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [4]:
class DQN(nn.Module):
    def __init__(self, num_frames, num_actions):
        super(DQN, self).__init__()
        self.num_frames = num_frames
        self.num_actions = num_actions
        
        # Layers
        self.conv1 = nn.Conv2d(
            in_channels=num_frames,
            out_channels=16,
            kernel_size=8,
            stride=4,
            padding=2
            )
        self.conv2 = nn.Conv2d(
            in_channels=16,
            out_channels=32,
            kernel_size=4,
            stride=2,
            padding=1
            )
        self.fc1 = nn.Linear(
            in_features=3200,
            out_features=256,
            )
        self.fc2 = nn.Linear(
            in_features=256,
            out_features=num_actions,
            )
        
        # Activation Functions
        self.relu = nn.ReLU()
    
    def flatten(self, x):
        batch_size = x.size()[0]
        x = x.view(batch_size, -1)
        return x
    
    def forward(self, x):
        
        # Forward pass
        x = self.relu(self.conv1(x))  # In: (80, 80, 4)  Out: (20, 20, 16)
        x = self.relu(self.conv2(x))  # In: (20, 20, 16) Out: (10, 10, 32)
        x = self.flatten(x)           # In: (10, 10, 32) Out: (3200,)
        x = self.relu(self.fc1(x))    # In: (3200,)      Out: (256,)
        x = self.fc2(x)               # In: (256,)       Out: (4,)
        
        return x

In [24]:
class DQN(nn.Module):
    def __init__(self, num_frames, num_actions):
        super(DQN, self).__init__()
        self.num_frames = num_frames
        self.num_actions = num_actions
        
        # Layers
        self.conv1 = nn.Conv2d(
            in_channels=num_frames,
            out_channels=16,
            kernel_size=8,
            stride=4,
            padding=2
            )
        self.conv2 = nn.Conv2d(
            in_channels=16,
            out_channels=4,
            kernel_size=1,
            stride=2,
            padding=1
            )
        
        # Dueling branches
        self.fc_value = nn.Linear(576, 32)
        self.fc_advantage = nn.Linear(576, 32)
        
        self.value = nn.Linear(32, 1)
        self.advantage = nn.Linear(32, num_actions)
        
        # Activation Functions
        self.relu = nn.ReLU()
    
    def flatten(self, x):
        batch_size = x.size()[0]
        x = x.view(batch_size, -1)
        return x
    
    def forward(self, x):
        # Forward pass
        x = self.relu(self.conv1(x))  # In: (80, 80, 4)  Out: (20, 20, 16)
        x = self.relu(self.conv2(x))  # In: (20, 20, 16) Out: (10, 10, 32)
        x = self.flatten(x)           # In: (10, 10, 32) Out: (3200,)
        
        value = self.relu(self.fc_value(x))    # In: (3200,)      Out: (256,)
        value = self.value(value)              # In: (256,)       Out: (1,)
        
        advantage = self.relu(self.fc_advantage(x))    # In: (3200,)      Out: (256,)
        advantage = self.advantage(advantage)          # In: (256,)       Out: (num_actions,)
        
        q_values = value + advantage - advantage.mean(dim=1, keepdim=True)
        
        return q_values


In [25]:
policy_net

DQN(
  (conv1): Conv2d(4, 16, kernel_size=(8, 8), stride=(4, 4), padding=(2, 2))
  (conv2): Conv2d(16, 4, kernel_size=(1, 1), stride=(2, 2), padding=(1, 1))
  (fc_value): Linear(in_features=576, out_features=32, bias=True)
  (fc_advantage): Linear(in_features=576, out_features=32, bias=True)
  (value): Linear(in_features=32, out_features=1, bias=True)
  (advantage): Linear(in_features=32, out_features=6, bias=True)
  (relu): ReLU()
)

In [26]:
num_frames=4
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 1
EPS_END = 0.1
EPS_DECAY = 1000000
TAU = 0.005
LR = 1e-4
UPDATE_FREQUENCY=4
replay_memory_size = int(1000000/80) #must be less than original paper due to memory constraints on datahub
replay_start_size=int(50000/80)
UPDATE_TARGET_NET_EVERY=10000
MIN_SQUARED_GRAD=0.01

# Get number of actions from gym action space
n_actions = env.action_space.n
# Get the number of state observations
state, info = env.reset()
n_observations = len(state)

policy_net = DQN(num_frames, n_actions).to(device)
target_net = DQN(num_frames, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

#optimizer = torch.optim.RMSprop(policy_net.parameters(), lr=LR, momentum=0.95,eps=MIN_SQUARED_GRAD)
optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(replay_memory_size)


steps_done = 0


def select_action(state,no_rand=False):
    global steps_done
    sample = random.random()
    eps_threshold = max(EPS_START + (EPS_START - EPS_END) * \
        (-1. * steps_done / EPS_DECAY), EPS_END)
    steps_done += 1
    if (no_rand or sample > eps_threshold) and itters > replay_start_size:
        with torch.no_grad():
            #print("not random")
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            #print(policy_net(state))
            pns = policy_net(state)
            return pns.max(1).indices.view(1, 1),pns.max().item()
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long), None


episode_durations = []


def plot_durations(episode_durations, show_result=False,y_axis_name="Score"):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel(y_axis_name)
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    window_size = 100
    if len(durations_t) >= window_size:
        # Calculate the rolling mean for every point
        means = torch.zeros(len(durations_t))
        for i in range(window_size - 1, len(durations_t)):
            means[i] = durations_t[i - window_size + 1:i + 1].mean()
        # Plot the rolling mean
        plt.plot(means.numpy())

        
    episodes = range(len(episode_durations))
    plt.xticks(episodes[::max(1, len(episodes)//10)])
    
    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [27]:
from skimage.color import rgb2gray

def optimize_model(num_updates):
    global target_net
    if len(memory) < BATCH_SIZE:
        return
    if 1==0 and num_updates%UPDATE_TARGET_NET_EVERY == 0:
        try:
            del target_net
        except:
            pass
        target_net = copy.deepcopy(policy_net).to(device)
        for p in target_net.parameters():
            p.requires_grad = False
            
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()
    
    
    
#def process(state,device):
   # state = rgb2gray(state[35:195, :, :])#state[0][35:195, :])#
    #state = rescale(state, scale=0.5)
    #print(state.shape)
    #state = state[np.newaxis, np.newaxis, :, :]
    
    #return torch.tensor(state, device=device, dtype=torch.float)

def preprocess(observation,device):
    observation = cv2.cvtColor(cv2.resize(observation, (84, 100)), cv2.COLOR_BGR2GRAY)
    observation = observation[7:91,:]
    #ret, observation = cv2.threshold(observation,1,255,cv2.THRESH_BINARY)
    state= np.reshape(observation,(84,84))
    #print(state.shape)
    state = state[np.newaxis, np.newaxis, :, :]
    #print(state.shape)
    return torch.tensor(state, device=device, dtype=torch.float)


In [28]:
import os
def save(model, t,avg_qvals,rewards):
    save_path = os.path.join('ckpt', 'model-{}'.format(t))
    torch.save(model.state_dict(), save_path)
    np.savez(os.path.join('ckpt', 'graphs-{}.npz'.format(t)), avg_qvals=avg_qvals, rewards=rewards)
    
def load(policy_net,ckptdir='ckpt'):
    global rewards
    global avg_qvals
    ckpts = [file for file in os.listdir(ckptdir) if 'model' in file]
    steps = [int(re.search('\d+', file).group(0)) for file in ckpts]
        
    latest_ckpt = ckpts[np.argmax(steps)]
    t = np.max(steps)
        
    print("Loading checkpoint: {}".format(latest_ckpt))
        
    policy_net.load_state_dict(torch.load(os.path.join(ckptdir, latest_ckpt)))
    
    with np.load(os.path.join(ckptdir, 'graphs-'+latest_ckpt[6:])+'.npz') as data:
        avg_qvals = data['avg_qvals'].tolist()
        rewards = data['rewards'].tolist()
    return t

In [29]:
rewards = []
avg_qvals = []

In [ ]:
####THIS IS THE MAIN TRAINING CODE
itters=0
#itters=load(policy_net)
select_action_every_N_frames = 1
from skimage.color import rgb2gray
from skimage.transform import rescale
import numpy as np
from tqdm import tqdm_notebook as tqdm
num_frames=4
num_updates=0
if torch.cuda.is_available():
    num_episodes = 16000
else:
    num_episodes = 50

try:
    progress_bar = tqdm(range(num_episodes), unit='episode')
    for i_episode in progress_bar:
        # Initialize the environment and get its state
        state, info = env.reset()
        #state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        state = preprocess(state,device)
        reward_for_this_episode=0
        avg_qval_for_this_episode=0
        qvals_added=1
        for t in count():
            itters+=1
            while state.size()[1] < num_frames:
                action = 1 # Fire
                #observation, reward, terminated, truncated, info
                new_frame, reward, done,truncated, info = env.step(action)
                new_frame = preprocess(new_frame,device)
                state = torch.cat([state, new_frame], 1)
            if action ==1:
                action,Qval = select_action(state)
            #print(state.shape)
            if itters%select_action_every_N_frames == 0:
                action, Qval = select_action(state)
                if Qval is not None:
                    avg_qval_for_this_episode+=Qval
                    qvals_added+=1
            observation, reward, terminated, truncated, _ = env.step(action.item())
            new_frame = preprocess(observation,device)
            new_state = torch.cat([state, new_frame], 1)
            new_state = new_state[:, 1:, :, :]

            reward = torch.tensor([reward], device=device)
            done = terminated or truncated

            if terminated:
                new_state = None
            #else:
             #  next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

            # Store the transition in memory
            memory.push(state, action, new_state, reward)
            reward_for_this_episode += reward
            # Move to the next state
            #tate = next_state
            state = new_state
            if itters%(UPDATE_FREQUENCY*select_action_every_N_frames)==0 and itters > replay_start_size :
                # Perform one step of the optimization (on the policy network)
                optimize_model(num_updates)
                num_updates+=1

                # Soft update of the target network's weights
                # θ′ ← τ θ + (1 −τ )θ′
                target_net_state_dict = target_net.state_dict()
                policy_net_state_dict = policy_net.state_dict()
                for key in policy_net_state_dict:
                    target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
                target_net.load_state_dict(target_net_state_dict)
            if itters % 5000 == 0:
                        progress_bar.set_description("t = {}, eps = {}, updating = {}".format(itters,max(EPS_START + (EPS_START - EPS_END) * \
        (-1 * itters / EPS_DECAY),EPS_END), itters > replay_start_size))

            if done:
                avg_qval_for_this_episode = avg_qval_for_this_episode/qvals_added
                avg_qvals.append(avg_qval_for_this_episode)
                #episode_durations.append(reward_for_this_episode)
                rewards.append(reward_for_this_episode.cpu())
                #plot_durations()
                break
        if i_episode % 50 == 0 and i_episode != 0:
            avg_return = np.mean(rewards[-50:])
            avg_qval = np.mean(avg_qvals[-50:])
            print("Average score (last 50 episodes): {:.2f}, avg qval: {:.2f}".format(avg_return,avg_qval))
except KeyboardInterrupt:
    if 1==1:
        print("Saving model before quitting...")
        
    env.close()
save(policy_net,itters,avg_qvals,rewards)
print('Complete')
plot_durations(rewards,show_result=True,y_axis_name="Average Score Per Episode")
plt.ioff()
plt.show()
plot_durations(avg_qvals,show_result=True,y_axis_name="Average Action Value (Q)")
plt.ioff()
plt.show()
np.save

/tmp/ipykernel_94/2399161922.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  progress_bar = tqdm(range(num_episodes), unit='episode')


  0%|          | 0/16000 [00:00<?, ?episode/s]

Average score (last 50 episodes): 157.00, avg qval: 3.11
Average score (last 50 episodes): 152.70, avg qval: 4.56
Average score (last 50 episodes): 133.70, avg qval: 4.50
Average score (last 50 episodes): 157.50, avg qval: 4.26
Average score (last 50 episodes): 168.40, avg qval: 3.68
Average score (last 50 episodes): 152.30, avg qval: 3.46
Average score (last 50 episodes): 152.40, avg qval: 3.25
Average score (last 50 episodes): 175.10, avg qval: 3.00
Average score (last 50 episodes): 188.50, avg qval: 2.97
Average score (last 50 episodes): 163.00, avg qval: 2.70
Average score (last 50 episodes): 183.40, avg qval: 2.65
Average score (last 50 episodes): 204.50, avg qval: 2.69
Average score (last 50 episodes): 179.00, avg qval: 3.05
Average score (last 50 episodes): 177.40, avg qval: 2.71
Average score (last 50 episodes): 176.60, avg qval: 2.76
Average score (last 50 episodes): 202.20, avg qval: 2.82
Average score (last 50 episodes): 189.90, avg qval: 3.27
Average score (last 50 episodes

In [7]:
##Visualize one game of the network playing

import time
from skimage.color import rgb2gray
from skimage.transform import rescale
import numpy as np
num_frames=4
render=True
see_what_agent_sees=True

if torch.cuda.is_available():
    num_episodes = 600
else:
    num_episodes = 50

for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    state, info = env.reset()
    #state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    state2 = preprocess(state,device)
    state = process(state,device)
    
    #print(state[0])
    if render:
        if see_what_agent_sees:
            img = plt.imshow(state2[0][0].cpu().numpy())
        else:
            img = plt.imshow(env.render())
    for t in count():
        if render and t%10==0:
            if see_what_agent_sees:
                img = plt.imshow(state2[0][0].cpu().numpy())
            else:
                img = plt.imshow(env.render())
            display.display(plt.gcf())
            display.clear_output(wait=True)
        
        while state.size()[1] < num_frames:
            action = 1 # Fire
            #observation, reward, terminated, truncated, info
            new_frame, reward, done,truncated, info = env.step(action)
            new_frame = process(new_frame,device)
            state = torch.cat([state, new_frame], 1)
                        
        #print(state.shape)                
        action = select_action(state,no_rand=True)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        new_frame = process(observation,device)
        nf2 = preprocess(observation,device)
        state2 = torch.cat([state2, nf2], 1)
        state2 = state2[:, 1:, :, :]
        new_state = torch.cat([state, new_frame], 1)
        new_state = new_state[:, 1:, :, :]
        
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            new_state = None
        #else:
         #  next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, new_state, reward)

        # Move to the next state
        #tate = next_state
        state = new_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break

print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()

/opt/conda/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


NameError: name 'preprocess' is not defined

In [ ]:
import time
from skimage.color import rgb2gray
from skimage.transform import rescale
import numpy as np
num_frames=4
env = gym.make("SpaceInvaders-v0", render_mode="rgb_array")
render=True
see_what_agent_sees=False

if torch.cuda.is_available():
    num_episodes = 600
else:
    num_episodes = 50

for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    state, info = env.reset()
    #state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    state = process(state,device)
    print(state[0])
    if render:
        if see_what_agent_sees:
            img = plt.imshow(state[0][0].cpu().numpy())
        else:
            img = plt.imshow(env.render())
    for t in count():
        if render:
            if see_what_agent_sees:
                img = plt.imshow(state[0][0].cpu().numpy())
            else:
                img = plt.imshow(env.render())
            display.display(plt.gcf())
            display.clear_output(wait=True)
        
        while state.size()[1] < num_frames:
            action = 1 # Fire
            #observation, reward, terminated, truncated, info
            new_frame, reward, done,truncated, info = env.step(action)
            new_frame = process(new_frame,device)
            state = torch.cat([state, new_frame], 1)
                        
        #print(state.shape)                
        action = select_action(state,no_rand=True)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        new_frame = process(observation,device)
        new_state = torch.cat([state, new_frame], 1)
        new_state = new_state[:, 1:, :, :]
        
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            new_state = None
        #else:
         #  next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, new_state, reward)

        # Move to the next state
        #tate = next_state
        state = new_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break

print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()

In [ ]:
#!pip install gym[classic_control]